Hi this is "Abhiram Bhaskar Kakarla ID: 219494693"




In this coding part , i have tried to aplly federation learning model to MNIST dataset

AS we know MNIST dataset is an image handwritten dataset, our objective in this is to correctly identify handwritten dataset
First, by using tensor flow  and federated learning i have built a model for training the data in local host

Afterwards, by using federated learning i have built server, client model and try to train model based on all clients

From the results, it is clearl that federated model with all clients runing improves the model performance by keeping the privacy of client as well.

REF:  ThIS is a sample code implementation for the paper "Communication Efficient learning of deep networks from decentralized data, H Brendan McMahn et al." 




In [ ]:
! pip install --upgrade tensorflow_federated

     |████████████████████████████████| 583kB 7.7MB/s 
     |████████████████████████████████| 706kB 46.5MB/s 
     |████████████████████████████████| 174kB 48.2MB/s 
     |████████████████████████████████| 194kB 46.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: attrs 20.3.0
    Uninstalling attrs-20.3.0:
      Successfully uninstalled attrs-20.3.0
  Found existing installation: cachetools 4.2.1
    Uninstalling cachetools-4.2.1:
      Successfully uninstalled cachetools-4.2.1


In [ ]:
pip install tensorflow-federated==0.4.0

     |████████████████████████████████| 204kB 8.2MB/s 
     |████████████████████████████████| 110.5MB 96kB/s 
     |████████████████████████████████| 512kB 39.0MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 3.8MB 43.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=db7b1b9da080ae066e37d88f2bcd922bca3937a4c705b2d47df846cb75ce97e8
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 1.15.5 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow-privacy 0.5.2 has requirement tensorflow-estimator>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:

In [ ]:
from __future__ import absolute_import, division, print_function

import collections

import numpy as np

import six.moves

from six.moves import range 

import tensorflow as tf

from tensorflow_federated import python as tff

tf.compat.v1.enable_v2_behavior()


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
@tff.federated_computation   #  this is run in tensorflow federated platform from google

def hello_world():
  return " hello this works"
hello_world()

' hello this works'

In [ ]:
mnist_train,mnist_test = tf.keras.datasets.mnist.load_data()
[(x.dtype,x.shape) for x in mnist_train]

11493376/11490434 [==============================] - 0s 0us/step


[(dtype('uint8'), (60000, 28, 28)), (dtype('uint8'), (60000,))]

In [ ]:
NUM_EXAMPLES_PER_USER = 1000  # image classification data ( mnist data has images like 0, here it is taken as first node, 1 second node --- non id dataset )

BATCH_SIZE = 100

def get_data_for_digit(source,digit):
  output_sequence = []
  all_samples = [ i for i, d in enumerate (source[1]) if d== digit]
  for i in range(0, min(len(all_samples), NUM_EXAMPLES_PER_USER), BATCH_SIZE):
    batch_samples = all_samples[i:i + BATCH_SIZE]
    output_sequence.append({ "x": np.array([ source[0][i].flatten()/255.0 for i in batch_samples], dtype=np.float32), "y": np.array([source[1][i] for i in batch_samples], dtype=np.int32)})
  print(output_sequence)
  return output_sequence

federated_test_data = [get_data_for_digit(mnist_test,d) for d in range(10)]

federated_train_data = [get_data_for_digit(mnist_train,d) for d in range(10)]

federated_train_data[5]

mnist_test[0].shape


[{'x': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), 'y': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)}, {'x': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), 'y': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

(10000, 28, 28)

In [ ]:
Batch_TYPE = tff.NamedTupleType([("x", tff.TensorType(tf.float32, [None, 784])), ("y", tff.TensorType(tf.int32,[None]))])

str(Batch_TYPE)  # dataset is changed to batch type here

'<x=float32[?,784],y=int32[?]>'

In [ ]:
MODEL_TYPE = tff.NamedTupleType([("weights", tff.TensorType(tf.float32,[784,10])), ("bias", tff.TensorType(tf.float32,[10]))])
str(MODEL_TYPE)                                                                                                                     # build the model here

'<weights=float32[784,10],bias=float32[10]>'

In [ ]:
@tff.tf_computation(MODEL_TYPE,Batch_TYPE)    # for loss computation
def batch_loss(model,batch):
  predicted_y=tf.nn.softmax(tf.matmul(batch.x, model.weights)+ model.bias)
  return -tf.reduce_mean(tf.reduce_sum(tf.one_hot(batch.y,10)*tf.log(predicted_y), reduction_indices=[1]))

In [ ]:
 initial_model ={ "weights": np.zeros([784,10], dtype=np.float32), "bias": np.zeros([10], dtype=np.float32) }
 sample_batch=federated_train_data[5][-1]
 batch_loss(initial_model, sample_batch)  # this is for local client data

2.3025854

In [ ]:
 learning_rate = 0.1
 str(batch_loss.type_signature)
 federated_train_data[0][0]["x"]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
@tff.tf_computation(MODEL_TYPE,Batch_TYPE,tf.float32)  # this is for all batches, using optimizer for minimizing loses

def batch_train(initial_model,batch,learning_rate):
  model_vars=tff.utils.get_variables("v", MODEL_TYPE)
  init_model=tff.utils.assign(model_vars,initial_model)
  optimizer=tf.train.GradientDescentOptimizer(learning_rate)
  with tf.control_dependencies([init_model]):
    train_model=optimizer.minimize(batch_loss(model_vars,batch))
  with tf.control_dependencies([train_model]):
    return tff.utils.identity(model_vars)
batch_train(initial_model, sample_batch, learning_rate )



Instructions for updating:
If using Keras pass *_constraint arguments to layers.



AnonymousTuple([(weights, array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)), (bias, array([-0.01      , -0.01      , -0.01      , -0.01      , -0.01      ,
        0.09000003, -0.01      , -0.01      , -0.01      , -0.01      ],
      dtype=float32))])

In [ ]:
str(batch_train.type_signature)

'(<<weights=float32[784,10],bias=float32[10]>,<x=float32[?,784],y=int32[?]>,float32> -> <weights=float32[784,10],bias=float32[10]>)'

In [ ]:
model = initial_model
losses = []
for _ in range(5):
  model= batch_train(model,sample_batch, 0.1)
  losses.append(batch_loss(model,sample_batch))
losses

[0.19690023, 0.13176315, 0.10113225, 0.08273812, 0.07030139]

In [ ]:
LOCAL_DATA_TYPE = tff.SequenceType(Batch_TYPE)    # locally train model built here

@tff.federated_computation(MODEL_TYPE,tf.float32,LOCAL_DATA_TYPE)
def local_train(initial_model, learning_rate, all_batches):
  @tff.federated_computation(MODEL_TYPE,Batch_TYPE)
  def batch_fn(model,batch):
    return batch_train(model,batch,learning_rate)
  return tff.sequence_reduce(all_batches,initial_model,batch_fn)
locally_trained_model= local_train(initial_model, learning_rate, federated_train_data[5])

In [ ]:
@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)

def local_eval(model, all_batches):
  return tff.sequence_sum(tff.sequence_map( tff.federated_computation(lambda b: batch_loss (model, b), Batch_TYPE), all_batches))
local_eval(initial_model, federated_train_data[5])

23.025854

In [ ]:
print(" initial model loss=", local_eval(initial_model, federated_train_data[5]))

print(" locally trained model loss=", local_eval(locally_trained_model, federated_train_data[5]))

 initial model loss= 23.025854
 locally trained model loss= 0.4348469


In [ ]:
# from the dataset instead passing 5 , for which we have trained the model try passing 2

print(" initial model loss=", local_eval(initial_model, federated_train_data[2]))

print(" locally trained model loss=", local_eval(locally_trained_model, federated_train_data[2]))

 initial model loss= 23.025854
 locally trained model loss= 60.698753


**As you can see there is high loss as we have trained data only for number 5 locally.**

In [ ]:
SERVER_MODEL_TYPE = tff.FederatedType(MODEL_TYPE, tff.SERVER, all_equal=True)  #server has created
CLIENT_DATA_TYPE = tff.FederatedType(LOCAL_DATA_TYPE, tff.CLIENTS)

In [ ]:
@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval(model,data):
  return tff.federated_mean(tff.federated_map(local_eval, [tff.federated_broadcast(model),data]))

In [ ]:
str(federated_eval.type_signature )

'(<<weights=float32[784,10],bias=float32[10]>@SERVER,{<x=float32[?,784],y=int32[?]>*}@CLIENTS> -> float32@SERVER)'

In [ ]:
print("initial model loss=", federated_eval(initial_model,federated_train_data))
print("locally train model loss=", federated_eval(locally_trained_model,federated_train_data))

initial model loss= 23.025852
locally train model loss= 54.432625


# still there is high loss because model has not been trained with all clients


In [ ]:
SERVER_FLOAT_TYPE =tff.FederatedType(tf.float32,tff.SERVER,all_equal=True)

@tff.federated_computation(SERVER_MODEL_TYPE,SERVER_FLOAT_TYPE, CLIENT_DATA_TYPE)

def federated_train(model, learning_rate, data):
  return tff.federated_mean(tff.federated_map(local_train,[tff.federated_broadcast(model), tff.federated_broadcast(learning_rate), data]))

In [ ]:
str(federated_train.type_signature)

'(<<weights=float32[784,10],bias=float32[10]>@SERVER,float32@SERVER,{<x=float32[?,784],y=int32[?]>*}@CLIENTS> -> <weights=float32[784,10],bias=float32[10]>@SERVER)'

In [ ]:
model = initial_model

learning_rate = 0.1

for round_num in range(5):
  model= federated_train(model, learning_rate, federated_train_data)
  learning_rate= learning_rate * 0.9
  loss = federated_eval( model, federated_train_data)
  print("round{},loss{}".format(round_num,loss))

round0,loss21.60552406311035
round1,loss20.365678787231445
round2,loss19.27480125427246
round3,loss18.31110954284668
round4,loss17.45725440979004


In [ ]:
print("initial_model test loss=", federated_eval(initial_model, federated_test_data))
print("trained model test loss=", federated_eval(model,federated_test_data))

initial_model test loss= 22.795593
trained model test loss= 17.278767
